CONSTRUCTION D'UN MODELE DE MACHINE LEARNING POUR RECOMMANDATION DE FILMS

0- Import des modules

In [1]:
# Les modules nécessaires sont : Pandas, Numpy, scikitlearn

In [2]:
import numpy as np
import pandas as pd
import sklearn

1- Collecte de données

In [3]:
# Trouver les données pertinentes :
# Ici, nous utiliserons les données de la base TMDB en ajoutant les bases IMDB d'acteurs et réalisateurs pour donner plus d'éléments de comparaison à l'outil

In [4]:
file_TMDB = "C:\\Users\\nicol\\Desktop\\Data\\VSC\\repos\\Wild Code School\\projet\\P2 - IMDB\\Recommandation\\tmdb_full_clean.csv"
file_title_principals = "C:\\Users\\nicol\\Desktop\\Data\\VSC\\repos\\Wild Code School\\projet\\P2 - IMDB\\Recommandation\\title.principals.csv"
file_name = "C:\\Users\\nicol\\Desktop\\Data\\VSC\\repos\\Wild Code School\\projet\\P2 - IMDB\\Recommandation\\name.basics.csv"
file_crew = "C:\\Users\\nicol\\Desktop\\Data\\VSC\\repos\\Wild Code School\\projet\\P2 - IMDB\\Recommandation\\title.crew.csv"
file_akas = "C:\\Users\\nicol\\Desktop\\Data\\VSC\\repos\\Wild Code School\\projet\\P2 - IMDB\\Recommandation\\title_akas.csv"

TMDB = pd.read_csv(file_TMDB)
TITLE = pd.read_csv(file_title_principals)
NAME = pd.read_csv(file_name)
CREW = pd.read_csv(file_crew)
AKAS = pd.read_csv(file_akas)

In [5]:
TMDB.columns

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'production_companies_name',
       'production_companies_country'],
      dtype='object')

In [6]:
# Filtrage de la base en fonction des besoins de l'outil

In [7]:
TMDB = TMDB[TMDB['status'] == 'Released'].reset_index(drop=True) # filtrage de la base totale pour ne retenir que les films sortis

In [8]:
TMDB = TMDB[TMDB['video'] == 0].reset_index(drop=True) # filtrage de la base totale pour retirer les films sortis qu'en vidéo

In [9]:
TMDB = TMDB.drop(columns= 'adult')
TMDB = TMDB.drop(columns= 'backdrop_path')
TMDB = TMDB.drop(columns= 'budget')
TMDB = TMDB.drop(columns= 'homepage')
TMDB = TMDB.drop(columns= 'production_countries')
TMDB = TMDB.drop(columns= 'revenue')
TMDB = TMDB.drop(columns= 'spoken_languages')
TMDB = TMDB.drop(columns= 'tagline')
TMDB = TMDB.drop(columns= 'status')
TMDB = TMDB.drop(columns= 'video')

In [10]:
TMDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31064 entries, 0 to 31063
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   genres                        31064 non-null  object 
 1   id                            31064 non-null  int64  
 2   imdb_id                       31064 non-null  object 
 3   original_language             31064 non-null  object 
 4   original_title                31064 non-null  object 
 5   overview                      30189 non-null  object 
 6   popularity                    31064 non-null  float64
 7   poster_path                   30069 non-null  object 
 8   release_date                  30916 non-null  object 
 9   runtime                       31064 non-null  int64  
 10  title                         31064 non-null  object 
 11  vote_average                  31064 non-null  float64
 12  vote_count                    31064 non-null  int64  
 13  p

In [11]:
TITLE.columns

Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')

In [12]:
TITLE = TITLE[TITLE['ordering'].isin([1])].reset_index(drop=True) # On ne garde que la 1ere tête d'affiche max

In [13]:
TITLE = TITLE[TITLE['category'].isin(['actor','actress','director','producer','composer','writer'])].reset_index(drop=True) # On conserve les métiers importants pour la recommandation

In [14]:
TITLE = TITLE.drop(columns='ordering')
TITLE = TITLE.drop(columns='job')
TITLE = TITLE.drop(columns='characters')

In [15]:
TITLE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40548 entries, 0 to 40547
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tconst    40548 non-null  object
 1   nconst    40548 non-null  object
 2   category  40548 non-null  object
dtypes: object(3)
memory usage: 950.5+ KB


In [16]:
TMDB_complete = TMDB.merge(TITLE, how='inner', left_on= 'imdb_id', right_on= 'tconst')

In [17]:
TMDB_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27283 entries, 0 to 27282
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   genres                        27283 non-null  object 
 1   id                            27283 non-null  int64  
 2   imdb_id                       27283 non-null  object 
 3   original_language             27283 non-null  object 
 4   original_title                27283 non-null  object 
 5   overview                      26788 non-null  object 
 6   popularity                    27283 non-null  float64
 7   poster_path                   26813 non-null  object 
 8   release_date                  27220 non-null  object 
 9   runtime                       27283 non-null  int64  
 10  title                         27283 non-null  object 
 11  vote_average                  27283 non-null  float64
 12  vote_count                    27283 non-null  int64  
 13  p

In [18]:
NAME.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369004 entries, 0 to 369003
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   nconst             369004 non-null  object 
 1   primaryName        369004 non-null  object 
 2   birthYear          86091 non-null   float64
 3   deathYear          12377 non-null   float64
 4   primaryProfession  361846 non-null  object 
 5   knownForTitles     368194 non-null  object 
dtypes: float64(2), object(4)
memory usage: 16.9+ MB


In [19]:
NAME.columns

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')

In [20]:
NAME = NAME.drop(columns='birthYear')
NAME = NAME.drop(columns='deathYear')
NAME = NAME.drop(columns='primaryProfession')
NAME = NAME.drop(columns='knownForTitles')

In [21]:
NAME.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369004 entries, 0 to 369003
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   nconst       369004 non-null  object
 1   primaryName  369004 non-null  object
dtypes: object(2)
memory usage: 5.6+ MB


In [22]:
TMDB_complete = TMDB_complete.merge(NAME, how='inner', on='nconst')

In [23]:
TMDB_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27282 entries, 0 to 27281
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   genres                        27282 non-null  object 
 1   id                            27282 non-null  int64  
 2   imdb_id                       27282 non-null  object 
 3   original_language             27282 non-null  object 
 4   original_title                27282 non-null  object 
 5   overview                      26787 non-null  object 
 6   popularity                    27282 non-null  float64
 7   poster_path                   26812 non-null  object 
 8   release_date                  27219 non-null  object 
 9   runtime                       27282 non-null  int64  
 10  title                         27282 non-null  object 
 11  vote_average                  27282 non-null  float64
 12  vote_count                    27282 non-null  int64  
 13  p

In [24]:
CREW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45681 entries, 0 to 45680
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tconst     45681 non-null  object
 1   directors  45330 non-null  object
 2   writers    40387 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [25]:
CREW = CREW.drop(columns='writers')

In [26]:
CREW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45681 entries, 0 to 45680
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tconst     45681 non-null  object
 1   directors  45330 non-null  object
dtypes: object(2)
memory usage: 713.9+ KB


In [27]:
TMDB_complete = TMDB_complete.merge(CREW, how='inner', on='tconst')

In [28]:
TMDB_complete = TMDB_complete.merge(NAME, how='left', left_on='directors', right_on='nconst') # On récupère le nom du réalisateur sur la table Name

In [29]:
AKAS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49187 entries, 0 to 49186
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleId          49187 non-null  object
 1   ordering         49187 non-null  int64 
 2   title            49187 non-null  object
 3   region           49187 non-null  object
 4   language         95 non-null     object
 5   types            41917 non-null  object
 6   attributes       1288 non-null   object
 7   isOriginalTitle  49187 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 2.7+ MB


In [30]:
AKAS = AKAS.drop(columns= 'ordering')
AKAS = AKAS.drop(columns= 'region')
AKAS = AKAS.drop(columns= 'language')
AKAS = AKAS.drop(columns= 'types')
AKAS = AKAS.drop(columns= 'attributes')
AKAS = AKAS.drop(columns= 'isOriginalTitle')

In [31]:
AKAS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49187 entries, 0 to 49186
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   titleId  49187 non-null  object
 1   title    49187 non-null  object
dtypes: object(2)
memory usage: 768.7+ KB


In [32]:
TMDB_complete = TMDB_complete.merge(AKAS, how='inner',left_on='tconst',right_on='titleId')

In [33]:
TMDB_complete.columns

Index(['genres', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'release_date', 'runtime',
       'title_x', 'vote_average', 'vote_count', 'production_companies_name',
       'production_companies_country', 'tconst', 'nconst_x', 'category',
       'primaryName_x', 'directors', 'nconst_y', 'primaryName_y', 'titleId',
       'title_y'],
      dtype='object')

In [34]:
TMDB_complete = TMDB_complete.drop(columns='tconst')
TMDB_complete = TMDB_complete.drop(columns='nconst_x')
TMDB_complete = TMDB_complete.drop(columns='directors')
TMDB_complete = TMDB_complete.drop(columns='nconst_y')
TMDB_complete = TMDB_complete.drop(columns='title_x')
TMDB_complete = TMDB_complete.drop(columns='titleId')

In [35]:
TMDB_complete = TMDB_complete.rename(columns= {'primaryName_x':'Actor_Name','primaryName_y':'DirectorName', 'title_y':'Titre_FR'})

In [36]:
TMDB_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29943 entries, 0 to 29942
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   genres                        29943 non-null  object 
 1   id                            29943 non-null  int64  
 2   imdb_id                       29943 non-null  object 
 3   original_language             29943 non-null  object 
 4   original_title                29943 non-null  object 
 5   overview                      29417 non-null  object 
 6   popularity                    29943 non-null  float64
 7   poster_path                   29444 non-null  object 
 8   release_date                  29880 non-null  object 
 9   runtime                       29943 non-null  int64  
 10  vote_average                  29943 non-null  float64
 11  vote_count                    29943 non-null  int64  
 12  production_companies_name     29943 non-null  object 
 13  p

2- Prétraitement de la donnée

In [37]:
# Nettoyage de la base : 
# - Mise en format 

In [38]:
TMDB_complete['genres'] = TMDB_complete['genres'].str.replace(r"[\[\]']", "", regex=True)
TMDB_complete['genres'] = TMDB_complete['genres'].str.split(', ')

TMDB_complete_expl = TMDB_complete['genres'].explode()
TMDB_complete_GD = pd.get_dummies(TMDB_complete_expl, columns='genre')

TMDB_complete = pd.concat([TMDB_complete, TMDB_complete_GD],axis=1).reset_index(drop=True)

TMDB_complete = TMDB_complete.drop(columns='genres')

In [39]:
TMDB_complete.columns

Index(['id', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'release_date', 'runtime', 'vote_average',
       'vote_count', 'production_companies_name',
       'production_companies_country', 'category', 'Actor_Name',
       'DirectorName', 'Titre_FR', '', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western'],
      dtype='object')

In [40]:
TMDB_complete['production_companies_name'] = TMDB_complete['production_companies_name'].str.replace(r"[\[\]']", "", regex=True)
TMDB_complete['production_companies_name'] = TMDB_complete['production_companies_name'].str.split(', ')

TMDB_complete[['prod_1', 'prod_2', 'prod_3']] = pd.DataFrame(TMDB_complete['production_companies_name'].apply(lambda x: x[:3]).tolist())

TMDB_complete = TMDB_complete.drop(columns='production_companies_name')
TMDB_complete = TMDB_complete.drop(columns='prod_2')
TMDB_complete = TMDB_complete.drop(columns='prod_3')

In [41]:
TMDB_complete.head()

,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,runtime,vote_average,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,prod_1
0,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,18.734,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,98,5.758,...,False,False,False,False,False,False,False,False,False,Miramax
1,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,18.734,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,98,5.758,...,False,False,False,False,False,False,False,False,False,Miramax
2,12,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",68.898,/5lc6nQc0VhWFYFbNv016xze8Jvy.jpg,2003-05-30,100,7.823,...,False,False,False,False,False,False,False,False,False,Pixar
3,12,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",68.898,/5lc6nQc0VhWFYFbNv016xze8Jvy.jpg,2003-05-30,100,7.823,...,False,False,False,False,False,False,False,False,False,Pixar
4,14,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",26.384,/wby9315QzVKdW9BonAefg8jGTTb.jpg,1999-09-15,122,8.027,...,False,False,False,False,False,False,False,False,False,Jinks/Cohen Company


In [42]:
# - Retrait des duplicatas

In [43]:
TMDB_complete = TMDB_complete.drop_duplicates(subset='imdb_id').reset_index(drop=True)

In [44]:
TMDB_complete.head()

,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,runtime,vote_average,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,prod_1
0,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,18.734,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,98,5.758,...,False,False,False,False,False,False,False,False,False,Miramax
1,12,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",68.898,/5lc6nQc0VhWFYFbNv016xze8Jvy.jpg,2003-05-30,100,7.823,...,False,False,False,False,False,False,False,False,False,Pixar
2,14,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",26.384,/wby9315QzVKdW9BonAefg8jGTTb.jpg,1999-09-15,122,8.027,...,False,False,False,False,False,False,False,False,False,Jinks/Cohen Company
3,16,tt0168629,en,Dancer in the Dark,"Selma, a Czech immigrant on the verge of blind...",13.781,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,2000-06-30,141,7.881,...,False,False,False,False,False,False,False,False,False,Lantia Cinema & Audiovisivi
4,17,tt0411267,en,The Dark,"In an attempt to pull her family together, Adè...",7.178,/wZeBHVnCvaS2bwkb8jFQ0PwZwXq.jpg,2005-09-28,87,5.815,...,True,False,False,False,False,False,False,False,False,Constantin Film


In [45]:
TMDB_complete.columns

Index(['id', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'release_date', 'runtime', 'vote_average',
       'vote_count', 'production_companies_country', 'category', 'Actor_Name',
       'DirectorName', 'Titre_FR', '', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western', 'prod_1'],
      dtype='object')

In [46]:
TMDB_complete.loc[TMDB_complete['Western'] == True, 'Titre_FR']


639                               Hidalgo
651                            Open Range
1036                       3h10 pour Yuma
1531         Spirit: L'étalon des plaines
2006     Blueberry : L'Expérience secrète
                       ...               
25703                       Bone Tomahawk
26260                           Brimstone
26466                             Le duel
26473            Le Dernier des Guerriers
26636                         Comancheria
Name: Titre_FR, Length: 72, dtype: object

In [47]:
# - Gestions des valeurs manquantes

In [48]:
missing_values = TMDB_complete.isnull().sum()
print(missing_values)

id                                  0
imdb_id                             0
original_language                   0
original_title                      0
overview                          495
popularity                          0
poster_path                       470
release_date                       63
runtime                             0
vote_average                        0
vote_count                          0
production_companies_country    11593
category                            0
Actor_Name                          0
DirectorName                     2096
Titre_FR                            0
                                    0
Action                              0
Adventure                           0
Animation                           0
Comedy                              0
Crime                               0
Documentary                         0
Drama                               0
Family                              0
Fantasy                             0
History     

In [49]:
import datetime as dt
TMDB_complete['release_date'] = pd.to_datetime(TMDB_complete['release_date'])

In [50]:
TMDB_complete['release_date'] = TMDB_complete['release_date'].dropna()

In [51]:
TMDB_complete['overview'] = TMDB_complete['overview'].fillna('Description non disponible')
TMDB_complete['poster_path'] = TMDB_complete['poster_path'].fillna('Chemin non disponible')
TMDB_complete['production_companies_country'] = TMDB_complete['production_companies_country'].fillna('Donnée non disponible')
TMDB_complete['prod_1'] = TMDB_complete['prod_1'].replace("", "Donnée non disponible")

In [52]:
TMDB_complete.head()

,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,runtime,vote_average,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,prod_1
0,5,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,18.734,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,98,5.758,...,False,False,False,False,False,False,False,False,False,Miramax
1,12,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",68.898,/5lc6nQc0VhWFYFbNv016xze8Jvy.jpg,2003-05-30,100,7.823,...,False,False,False,False,False,False,False,False,False,Pixar
2,14,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",26.384,/wby9315QzVKdW9BonAefg8jGTTb.jpg,1999-09-15,122,8.027,...,False,False,False,False,False,False,False,False,False,Jinks/Cohen Company
3,16,tt0168629,en,Dancer in the Dark,"Selma, a Czech immigrant on the verge of blind...",13.781,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,2000-06-30,141,7.881,...,False,False,False,False,False,False,False,False,False,Lantia Cinema & Audiovisivi
4,17,tt0411267,en,The Dark,"In an attempt to pull her family together, Adè...",7.178,/wZeBHVnCvaS2bwkb8jFQ0PwZwXq.jpg,2005-09-28,87,5.815,...,True,False,False,False,False,False,False,False,False,Constantin Film


In [53]:
TMDB_complete.columns

Index(['id', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'release_date', 'runtime', 'vote_average',
       'vote_count', 'production_companies_country', 'category', 'Actor_Name',
       'DirectorName', 'Titre_FR', '', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western', 'prod_1'],
      dtype='object')

In [54]:
# remise en forme des colonnes et sauvegarde du fichier

colonne_nvOrdre = ['imdb_id','original_title','Titre_FR','DirectorName','Actor_Name','overview','runtime','release_date','prod_1','production_companies_country', 'vote_average','vote_count','popularity','poster_path','Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western','id']

RECO = TMDB_complete[colonne_nvOrdre]

RECO.to_csv("bdd_outil_reco.csv")

In [55]:
RECO.head()

,imdb_id,original_title,Titre_FR,DirectorName,Actor_Name,overview,runtime,release_date,prod_1,production_companies_country,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,id
0,tt0113101,Four Rooms,Groom Service,NaN,Tim Roth,It's Ted the Bellhop's first night on the job....,98,1995-12-09,Miramax,"['US', 'US']",...,False,False,False,False,False,False,False,False,False,5
1,tt0266543,Finding Nemo,Le Monde de Nemo,NaN,Albert Brooks,"Nemo, an adventurous young clownfish, is unexp...",100,2003-05-30,Pixar,['US'],...,False,False,False,False,False,False,False,False,False,12
2,tt0169547,American Beauty,American Beauty,Sam Mendes,Kevin Spacey,"Lester Burnham, a depressed suburban father in...",122,1999-09-15,Jinks/Cohen Company,"['', 'US']",...,False,False,False,False,False,False,False,False,False,14
3,tt0168629,Dancer in the Dark,Dancer in the Dark,Lars von Trier,Björk,"Selma, a Czech immigrant on the verge of blind...",141,2000-06-30,Lantia Cinema & Audiovisivi,"['', '', 'DK', 'DK', 'SE', 'FR', 'SE', 'FR', '...",...,False,False,False,False,False,False,False,False,False,16
4,tt0411267,The Dark,The Dark,John Fawcett,Sean Bean,"In an attempt to pull her family together, Adè...",87,2005-09-28,Constantin Film,"['DE', 'GB', 'GB', 'GB']",...,True,False,False,False,False,False,False,False,False,17


OPTIMISATION DE LA BASE DE DONNEES

In [56]:
RECO_TEST = RECO

In [57]:
Date_extraction = "1999-01-01"
VoteMin = 1500

In [58]:
RECO_TEST.head()

,imdb_id,original_title,Titre_FR,DirectorName,Actor_Name,overview,runtime,release_date,prod_1,production_companies_country,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,id
0,tt0113101,Four Rooms,Groom Service,NaN,Tim Roth,It's Ted the Bellhop's first night on the job....,98,1995-12-09,Miramax,"['US', 'US']",...,False,False,False,False,False,False,False,False,False,5
1,tt0266543,Finding Nemo,Le Monde de Nemo,NaN,Albert Brooks,"Nemo, an adventurous young clownfish, is unexp...",100,2003-05-30,Pixar,['US'],...,False,False,False,False,False,False,False,False,False,12
2,tt0169547,American Beauty,American Beauty,Sam Mendes,Kevin Spacey,"Lester Burnham, a depressed suburban father in...",122,1999-09-15,Jinks/Cohen Company,"['', 'US']",...,False,False,False,False,False,False,False,False,False,14
3,tt0168629,Dancer in the Dark,Dancer in the Dark,Lars von Trier,Björk,"Selma, a Czech immigrant on the verge of blind...",141,2000-06-30,Lantia Cinema & Audiovisivi,"['', '', 'DK', 'DK', 'SE', 'FR', 'SE', 'FR', '...",...,False,False,False,False,False,False,False,False,False,16
4,tt0411267,The Dark,The Dark,John Fawcett,Sean Bean,"In an attempt to pull her family together, Adè...",87,2005-09-28,Constantin Film,"['DE', 'GB', 'GB', 'GB']",...,True,False,False,False,False,False,False,False,False,17


In [59]:
RECO_TEST.columns

Index(['imdb_id', 'original_title', 'Titre_FR', 'DirectorName', 'Actor_Name',
       'overview', 'runtime', 'release_date', 'prod_1',
       'production_companies_country', 'vote_average', 'vote_count',
       'popularity', 'poster_path', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western', 'id'],
      dtype='object')

In [60]:
RECO_TEST.groupby('DirectorName').agg(count= ('DirectorName','size'), sum_VoteCount = ('vote_count','sum')).sort_values(by='sum_VoteCount',ascending=False).head()

,count,sum_VoteCount
DirectorName,,
Christopher Nolan,11,190408
Peter Jackson,9,118284
Steven Spielberg,19,115361
David Fincher,10,111360
David Yates,8,108207


In [61]:
import pandas as pd

# Étape 1 : Identifier les 50 réalisateurs les plus populaires
top_50_directors = (
    RECO_TEST.groupby('DirectorName')
    .agg(count=('DirectorName', 'size'), sum_VoteCount=('vote_count', 'sum'))
    .sort_values(by='sum_VoteCount', ascending=False)
    .head(50)
    .index  # Récupère les noms des réalisateurs
)

# Étape 2 : Filtrer uniquement les réalisateurs dans le top 50
RECO_TEST['DirectorName'] = RECO_TEST['DirectorName'].where(RECO_TEST['DirectorName'].isin(top_50_directors), other='Other')

# Étape 3 : Appliquer get_dummies()
director_dummies = pd.get_dummies(RECO_TEST['DirectorName'], prefix='Is')

# Étape 4 : Ajouter les colonnes au DataFrame principal
RECO_TEST = pd.concat([RECO_TEST, director_dummies], axis=1)

# Afficher les premières lignes pour vérifier
RECO_TEST.head()


C:\Users\nicol\AppData\Local\Temp\ipykernel_33640\216644067.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RECO_TEST['DirectorName'] = RECO_TEST['DirectorName'].where(RECO_TEST['DirectorName'].isin(top_50_directors), other='Other')


,imdb_id,original_title,Titre_FR,DirectorName,Actor_Name,overview,runtime,release_date,prod_1,production_companies_country,...,Is_Ron Howard,Is_Sam Mendes,Is_Sam Raimi,Is_Shawn Levy,Is_Steven Soderbergh,Is_Steven Spielberg,Is_Tim Burton,Is_Todd Phillips,Is_Wes Anderson,Is_Zack Snyder
0,tt0113101,Four Rooms,Groom Service,Other,Tim Roth,It's Ted the Bellhop's first night on the job....,98,1995-12-09,Miramax,"['US', 'US']",...,False,False,False,False,False,False,False,False,False,False
1,tt0266543,Finding Nemo,Le Monde de Nemo,Other,Albert Brooks,"Nemo, an adventurous young clownfish, is unexp...",100,2003-05-30,Pixar,['US'],...,False,False,False,False,False,False,False,False,False,False
2,tt0169547,American Beauty,American Beauty,Sam Mendes,Kevin Spacey,"Lester Burnham, a depressed suburban father in...",122,1999-09-15,Jinks/Cohen Company,"['', 'US']",...,False,True,False,False,False,False,False,False,False,False
3,tt0168629,Dancer in the Dark,Dancer in the Dark,Other,Björk,"Selma, a Czech immigrant on the verge of blind...",141,2000-06-30,Lantia Cinema & Audiovisivi,"['', '', 'DK', 'DK', 'SE', 'FR', 'SE', 'FR', '...",...,False,False,False,False,False,False,False,False,False,False
4,tt0411267,The Dark,The Dark,Other,Sean Bean,"In an attempt to pull her family together, Adè...",87,2005-09-28,Constantin Film,"['DE', 'GB', 'GB', 'GB']",...,False,False,False,False,False,False,False,False,False,False


In [62]:
RECO_TEST.shape

(27282, 85)

In [63]:
RECO_TEST.groupby('prod_1').agg(count= ('prod_1','size'), sum_VoteCount = ('vote_count','sum')).sort_values(by='sum_VoteCount',ascending=False).head(50)

,count,sum_VoteCount
prod_1,,
Marvel Studios,32,514528
Universal Pictures,206,511663
Warner Bros. Pictures,78,476231
Walt Disney Pictures,134,469317
Columbia Pictures,140,430536
Paramount,159,406049
New Line Cinema,131,330463
Pixar,20,248598
Summit Entertainment,40,190461


In [64]:
RECO_TEST.shape

(27282, 85)

In [65]:
import pandas as pd

# Étape 1 : Identifier les 50 réalisateurs les plus populaires
top_50_prod = (
    RECO_TEST.groupby('prod_1')
    .agg(count=('prod_1', 'size'), sum_VoteCount=('vote_count', 'sum'))
    .sort_values(by='sum_VoteCount', ascending=False)
    .head(50)
    .index  # Récupère les noms des réalisateurs
)

# Étape 2 : Filtrer uniquement les réalisateurs dans le top 50
RECO_TEST['prod_1'] = RECO_TEST['prod_1'].where(RECO_TEST['prod_1'].isin(top_50_prod), other='Other')

# Étape 3 : Appliquer get_dummies()
prod_dummies = pd.get_dummies(RECO_TEST['prod_1'], prefix='Is')

# Étape 4 : Ajouter les colonnes au DataFrame principal
RECO_TEST = pd.concat([RECO_TEST, prod_dummies], axis=1)

# Afficher les premières lignes pour vérifier
RECO_TEST.head()

,imdb_id,original_title,Titre_FR,DirectorName,Actor_Name,overview,runtime,release_date,prod_1,production_companies_country,...,Is_The Weinstein Company,Is_Thunder Road,Is_Touchstone Pictures,Is_TriStar Pictures,Is_Universal Pictures,Is_Village Roadshow Pictures,Is_Walt Disney Animation Studios,Is_Walt Disney Pictures,Is_Warner Bros. Pictures,Is_Working Title Films
0,tt0113101,Four Rooms,Groom Service,Other,Tim Roth,It's Ted the Bellhop's first night on the job....,98,1995-12-09,Miramax,"['US', 'US']",...,False,False,False,False,False,False,False,False,False,False
1,tt0266543,Finding Nemo,Le Monde de Nemo,Other,Albert Brooks,"Nemo, an adventurous young clownfish, is unexp...",100,2003-05-30,Pixar,['US'],...,False,False,False,False,False,False,False,False,False,False
2,tt0169547,American Beauty,American Beauty,Sam Mendes,Kevin Spacey,"Lester Burnham, a depressed suburban father in...",122,1999-09-15,Other,"['', 'US']",...,False,False,False,False,False,False,False,False,False,False
3,tt0168629,Dancer in the Dark,Dancer in the Dark,Other,Björk,"Selma, a Czech immigrant on the verge of blind...",141,2000-06-30,Other,"['', '', 'DK', 'DK', 'SE', 'FR', 'SE', 'FR', '...",...,False,False,False,False,False,False,False,False,False,False
4,tt0411267,The Dark,The Dark,Other,Sean Bean,"In an attempt to pull her family together, Adè...",87,2005-09-28,Other,"['DE', 'GB', 'GB', 'GB']",...,False,False,False,False,False,False,False,False,False,False


In [66]:
print(RECO_TEST.columns[85:137])

Index(['Is_"The Donners Company"', 'Is_20th Century Fox',
       'Is_Amblin Entertainment', 'Is_Blumhouse Productions',
       'Is_Castle Rock Entertainment', 'Is_Columbia Pictures', 'Is_DC Comics',
       'Is_DC Entertainment', 'Is_Dimension Films', 'Is_Donnée non disponible',
       'Is_DreamWorks Animation', 'Is_DreamWorks Pictures',
       'Is_Dune Entertainment', 'Is_EuropaCorp', 'Is_Focus Features',
       'Is_Fox 2000 Pictures', 'Is_Fox Searchlight Pictures', 'Is_Gaumont',
       'Is_Heyday Films', 'Is_Illumination', 'Is_Legendary Pictures',
       'Is_Lionsgate', 'Is_Lucasfilm Ltd.', 'Is_Marvel Enterprises',
       'Is_Marvel Studios', 'Is_Metro-Goldwyn-Mayer', 'Is_Miramax',
       'Is_New Line Cinema', 'Is_New Regency Pictures', 'Is_Original Film',
       'Is_Other', 'Is_Paramount', 'Is_Pixar', 'Is_RatPac Entertainment',
       'Is_Regency Enterprises', 'Is_Scott Free Productions', 'Is_Screen Gems',
       'Is_Skydance Media', 'Is_Studio Ghibli', 'Is_Summit Entertainment',
   

In [67]:
RECO_TEST.groupby('Actor_Name').agg(count= ('prod_1','size'), sum_VoteCount = ('vote_count','sum')).sort_values(by='sum_VoteCount',ascending=False).head(50)

,count,sum_VoteCount
Actor_Name,,
Leonardo DiCaprio,21,213340
Robert Downey Jr.,16,200895
Tom Hanks,29,176722
Daniel Radcliffe,19,170440
Johnny Depp,31,161439
Will Smith,25,160305
Brad Pitt,23,147347
Christian Bale,22,143305
Tom Cruise,24,142261


In [68]:
# Étape 1 : Identifier les 50 réalisateurs les plus populaires
top_100_actor = (
    RECO_TEST.groupby('Actor_Name')
    .agg(count=('Actor_Name', 'size'), sum_VoteCount=('vote_count', 'sum'))
    .sort_values(by='sum_VoteCount', ascending=False)
    .head(100)
    .index  # Récupère les noms des réalisateurs
)

# Étape 2 : Filtrer uniquement les réalisateurs dans le top 50
RECO_TEST['Actor_Name'] = RECO_TEST['Actor_Name'].where(RECO_TEST['Actor_Name'].isin(top_100_actor), other='Other')

# Étape 3 : Appliquer get_dummies()
Actor_Name_dummies = pd.get_dummies(RECO_TEST['Actor_Name'], prefix='Is')

# Étape 4 : Ajouter les colonnes au DataFrame principal
RECO_TEST = pd.concat([RECO_TEST, Actor_Name_dummies], axis=1)

# Afficher les premières lignes pour vérifier
RECO_TEST.head()

,imdb_id,original_title,Titre_FR,DirectorName,Actor_Name,overview,runtime,release_date,prod_1,production_companies_country,...,Is_Tobey Maguire,Is_Tom Cruise,Is_Tom Hanks,Is_Tom Hardy,Is_Tom Holland,Is_Tommy Lee Jones,Is_Uma Thurman,Is_Vin Diesel,Is_Will Ferrell,Is_Will Smith
0,tt0113101,Four Rooms,Groom Service,Other,Other,It's Ted the Bellhop's first night on the job....,98,1995-12-09,Miramax,"['US', 'US']",...,False,False,False,False,False,False,False,False,False,False
1,tt0266543,Finding Nemo,Le Monde de Nemo,Other,Other,"Nemo, an adventurous young clownfish, is unexp...",100,2003-05-30,Pixar,['US'],...,False,False,False,False,False,False,False,False,False,False
2,tt0169547,American Beauty,American Beauty,Sam Mendes,Kevin Spacey,"Lester Burnham, a depressed suburban father in...",122,1999-09-15,Other,"['', 'US']",...,False,False,False,False,False,False,False,False,False,False
3,tt0168629,Dancer in the Dark,Dancer in the Dark,Other,Other,"Selma, a Czech immigrant on the verge of blind...",141,2000-06-30,Other,"['', '', 'DK', 'DK', 'SE', 'FR', 'SE', 'FR', '...",...,False,False,False,False,False,False,False,False,False,False
4,tt0411267,The Dark,The Dark,Other,Other,"In an attempt to pull her family together, Adè...",87,2005-09-28,Other,"['DE', 'GB', 'GB', 'GB']",...,False,False,False,False,False,False,False,False,False,False


In [69]:
RECO_TEST.shape

(27282, 237)

In [70]:
print(RECO_TEST.columns[136:200])

Index(['Is_Adam Sandler', 'Is_Amy Adams', 'Is_Andrew Garfield',
       'Is_Angelina Jolie', 'Is_Anna Kendrick', 'Is_Anne Hathaway',
       'Is_Antonio Banderas', 'Is_Arnold Schwarzenegger', 'Is_Ben Affleck',
       'Is_Ben Stiller', 'Is_Benedict Cumberbatch', 'Is_Brad Pitt',
       'Is_Bradley Cooper', 'Is_Brendan Fraser', 'Is_Bruce Willis',
       'Is_Channing Tatum', 'Is_Chris Evans', 'Is_Chris Hemsworth',
       'Is_Chris Pine', 'Is_Chris Pratt', 'Is_Christian Bale',
       'Is_Colin Farrell', 'Is_Colin Firth', 'Is_Daisy Ridley',
       'Is_Daniel Craig', 'Is_Daniel Radcliffe', 'Is_Denzel Washington',
       'Is_Dwayne Johnson', 'Is_Dylan O'Brien', 'Is_Eddie Redmayne',
       'Is_Edward Norton', 'Is_Elijah Wood', 'Is_Emily Blunt',
       'Is_Ethan Hawke', 'Is_Ewan McGregor', 'Is_George Clooney',
       'Is_Gerard Butler', 'Is_Harrison Ford', 'Is_Hayden Christensen',
       'Is_Henry Cavill', 'Is_Hugh Jackman', 'Is_Jack Black', 'Is_Jackie Chan',
       'Is_Jake Gyllenhaal', 'Is_James

In [71]:
print(RECO_TEST.columns[201:237])

Index(['Is_Michael Fassbender', 'Is_Mike Myers', 'Is_Milla Jovovich',
       'Is_Morgan Freeman', 'Is_Naomi Watts', 'Is_Natalie Portman',
       'Is_Nicolas Cage', 'Is_Nicole Kidman', 'Is_Other', 'Is_Owen Wilson',
       'Is_Patrick Stewart', 'Is_Paul Rudd', 'Is_Robert De Niro',
       'Is_Robert Downey Jr.', 'Is_Robin Williams', 'Is_Russell Crowe',
       'Is_Ryan Gosling', 'Is_Ryan Reynolds', 'Is_Sam Worthington',
       'Is_Samuel L. Jackson', 'Is_Sandra Bullock', 'Is_Scarlett Johansson',
       'Is_Shailene Woodley', 'Is_Shia LaBeouf', 'Is_Steve Carell',
       'Is_Sylvester Stallone', 'Is_Tobey Maguire', 'Is_Tom Cruise',
       'Is_Tom Hanks', 'Is_Tom Hardy', 'Is_Tom Holland', 'Is_Tommy Lee Jones',
       'Is_Uma Thurman', 'Is_Vin Diesel', 'Is_Will Ferrell', 'Is_Will Smith'],
      dtype='object')


In [72]:
RECO_TEST.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27282 entries, 0 to 27281
Columns: 237 entries, imdb_id to Is_Will Smith
dtypes: bool(222), datetime64[ns](1), float64(2), int64(3), object(9)
memory usage: 8.9+ MB


Bonus reco

In [73]:
#Certains critères vont obtenir un bonus pour influencer la recommandation :
# genres : animation, Comedy, Family, Fantasy, Horror, Western, SciFi, Guerre
# boîtes de prod : disney, pixar, ghibli, Dreamworks, Lucasfilm, Marvel studios

In [74]:
# bonus pondération :

B_Animation = 1
B_Comedy = 1
B_Family = 1
B_Fantasy = 1
B_Horror = 1
B_SciFi = 1
B_Western = 1
B_war = 1

B_Disney = 1
B_Pixar = 1
B_Ghibli = 1
B_Dreamworks = 1
B_Lucasfilm = 1
B_Marvel = 2

In [75]:
def bonus_animation(x):
    if x == 1:
        return B_Animation
    else :
        return 0
RECO_TEST['bonus_animation'] = RECO_TEST['Animation'].apply(bonus_animation)

def bonus_Comedy(x):
    if x == 1:
        return B_Comedy
    else :
        return 0
RECO_TEST['bonus_Comedy'] = RECO_TEST['Comedy'].apply(bonus_Comedy)

def bonus_Family(x):
    if x == 1:
        return B_Family
    else :
        return 0
RECO_TEST['bonus_Family'] = RECO_TEST['Family'].apply(bonus_Family)

def bonus_Horror(x):
    if x == 1:
        return B_Horror
    else :
        return 0
RECO_TEST['bonus_Horror'] = RECO_TEST['Horror'].apply(bonus_Horror)

def bonus_Comedy(x):
    if x == 1:
        return B_Comedy
    else :
        return 0
RECO_TEST['bonus_Comedy'] = RECO_TEST['Comedy'].apply(bonus_Comedy)

def bonus_Scifi(x):
    if x == 1:
        return B_SciFi
    else :
        return 0
RECO_TEST['bonus_Scifi'] = RECO_TEST['Science Fiction'].apply(bonus_Scifi)

def bonus_War(x):
    if x == 1:
        return B_war
    else :
        return 0
RECO_TEST['bonus_War'] = RECO_TEST['War'].apply(bonus_War)

def bonus_Western(x):
    if x == 1:
        return B_Western
    else :
        return 0
RECO_TEST['bonus_Western'] = RECO_TEST['Western'].apply(bonus_Western)

def bonus_Disney(x):
    if x == 1:
        return B_Disney
    else :
        return 0
RECO_TEST['bonus_Disney'] = RECO_TEST['Is_Walt Disney Pictures'].apply(bonus_Disney)

def bonus_Pixar(x):
    if x == 1:
        return B_Pixar
    else :
        return 0
RECO_TEST['bonus_Pixar'] = RECO_TEST['Is_Pixar'].apply(bonus_Pixar)

def bonus_Dreamworks(x):
    if x == 1:
        return B_Dreamworks
    else :
        return 0
RECO_TEST['bonus_Dreamworks'] = RECO_TEST['Is_DreamWorks Pictures'].apply(bonus_Dreamworks)

def bonus_ghibli(x):
    if x == 1:
        return B_Ghibli
    else :
        return 0
RECO_TEST['bonus_Ghibli'] = RECO_TEST['Is_Studio Ghibli'].apply(bonus_ghibli)

def bonus_Lucasfilm(x):
    if x == 1:
        return B_Lucasfilm
    else :
        return 0
RECO_TEST['bonus_Lucasfilm'] = RECO_TEST['Is_Lucasfilm Ltd.'].apply(bonus_Lucasfilm)

def bonus_Marvel(x):
    if x == 1:
        return B_Marvel
    else :
        return 0
RECO_TEST['bonus_Marvel'] = RECO_TEST['Is_Marvel Studios'].apply(bonus_Marvel)

In [76]:
RECO_TEST.shape

(27282, 250)

In [77]:
# RECO_TEST = RECO_TEST[RECO_TEST['release_date'] >= Date_extraction ].reset_index(drop=True)
# RECO_TEST = RECO_TEST[RECO_TEST['vote_count'] >= VoteMin].reset_index(drop=True)

In [78]:
RECO_TEST.shape

(27282, 250)

3- Séparation des données

In [79]:
# Séparation de la base final : entrainement 80%/ test 20%

In [80]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [81]:
RECO = RECO_TEST

In [82]:
ColModel = [
    'vote_count',
    'vote_average',
    'DirectorName',
    'popularity',
    'Actor_Name',
    'prod_1',
    
    'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'History', 'Horror', 'Music', 'Mystery', 'Science Fiction', 'War', 'Western',

       'Is_Alfonso Cuarón',
       'Is_Antoine Fuqua', 'Is_Brad Bird', 'Is_Bryan Singer',
       'Is_Chris Columbus', 'Is_Christopher Nolan', 'Is_Clint Eastwood',
       'Is_Darren Aronofsky', 'Is_David Ayer', 'Is_David Fincher',
       'Is_David Yates', 'Is_Denis Villeneuve', 'Is_Doug Liman',
       'Is_Edgar Wright', 'Is_Francis Lawrence', 'Is_Gore Verbinski',
       'Is_Guillermo del Toro', 'Is_Guy Ritchie', 'Is_J.J. Abrams',
       'Is_James Cameron', 'Is_James Gunn', 'Is_James Mangold', 'Is_James Wan',
       'Is_Jon Favreau', 'Is_Jon Watts', 'Is_Joss Whedon', 'Is_Justin Lin',
       'Is_Kenneth Branagh', 'Is_Luc Besson', 'Is_M. Night Shyamalan',
       'Is_Marc Webb', 'Is_Martin Scorsese', 'Is_Matthew Vaughn',
       'Is_Michael Bay', 'Is_Other', 'Is_Peter Jackson', 'Is_Peyton Reed',
       'Is_Quentin Tarantino', 'Is_Ridley Scott', 'Is_Robert Zemeckis',
       'Is_Roland Emmerich', 'Is_Ron Howard', 'Is_Sam Mendes', 'Is_Sam Raimi',
       'Is_Shawn Levy', 'Is_Steven Soderbergh', 'Is_Steven Spielberg',
       'Is_Tim Burton', 'Is_Todd Phillips', 'Is_Wes Anderson',
       'Is_Zack Snyder',
       
       'Is_"The Donners Company"', 'Is_20th Century Fox', 'Is_Amblin Entertainment', 'Is_Blumhouse Productions', 'Is_Castle Rock Entertainment', 'Is_Columbia Pictures', 'Is_DC Comics', 'Is_DC Entertainment', 'Is_Dimension Films', 'Is_Donnée non disponible', 'Is_DreamWorks Animation', 'Is_DreamWorks Pictures', 'Is_Dune Entertainment', 'Is_EuropaCorp', 'Is_Focus Features', 'Is_Fox 2000 Pictures', 'Is_Fox Searchlight Pictures', 'Is_Heyday Films', 'Is_Legendary Pictures', 'Is_Lionsgate', 'Is_Lucasfilm Ltd.', 'Is_Marvel Enterprises', 'Is_Marvel Studios', 'Is_Metro-Goldwyn-Mayer', 'Is_Miramax', 'Is_New Line Cinema', 'Is_New Regency Pictures', 'Is_Original Film', 'Is_Other', 'Is_Paramount', 'Is_Pixar', 'Is_RatPac Entertainment', 'Is_Regency Enterprises', 'Is_Scott Free Productions', 'Is_Screen Gems', 'Is_Skydance Media','Is_Studio Ghibli', 'Is_Summit Entertainment', 'Is_TSG Entertainment', 'Is_The Weinstein Company',
       'Is_Thunder Road', 'Is_Touchstone Pictures', 'Is_TriStar Pictures', 'Is_Universal Pictures', 'Is_Village Roadshow Pictures', 'Is_Walt Disney Pictures', 'Is_Warner Bros. Pictures', 'Is_Working Title Films'
       
       ,'Is_Adam Sandler', 'Is_Amy Adams', 'Is_Andrew Garfield', 'Is_Angelina Jolie', 'Is_Anne Hathaway', 'Is_Antonio Banderas', 'Is_Arnold Schwarzenegger', 'Is_Ben Affleck', 'Is_Ben Stiller', 'Is_Benedict Cumberbatch', 'Is_Brad Pitt', 'Is_Bradley Cooper', 'Is_Brendan Fraser', 'Is_Bruce Willis', 'Is_Chris Evans', 'Is_Chris Hemsworth', 'Is_Chris Pine', 'Is_Chris Pratt', 'Is_Christian Bale', 'Is_Colin Farrell', 'Is_Colin Firth', 'Is_Daisy Ridley','Is_Daniel Craig', 'Is_Daniel Radcliffe', 'Is_Denzel Washington', 'Is_Dwayne Johnson', "Is_Dylan O'Brien", 'Is_Eddie Redmayne', 'Is_Edward Norton', 'Is_Elijah Wood', 'Is_Emily Blunt', 'Is_Ethan Hawke', 'Is_Ewan McGregor', 'Is_George Clooney', 'Is_Gerard Butler', 'Is_Harrison Ford', 'Is_Hayden Christensen', 'Is_Henry Cavill', 'Is_Hugh Jackman', 'Is_Jackie Chan', 'Is_Jake Gyllenhaal', 'Is_James Franco', 'Is_James McAvoy', 'Is_Jamie Foxx', 'Is_Jason Statham',
       'Is_Jennifer Lawrence', 'Is_Jesse Eisenberg', 'Is_Jim Carrey', 'Is_Joaquin Phoenix', 'Is_John Cusack', 'Is_Johnny Depp', 'Is_Joseph Gordon-Levitt', 'Is_Keanu Reeves', 'Is_Kevin Spacey', 'Is_Kristen Stewart', 'Is_Leonardo DiCaprio', 'Is_Liam Neeson', 'Is_Mark Wahlberg', 'Is_Matt Damon', 'Is_Matthew McConaughey', 'Is_Mel Gibson',
       'Is_Michael Fassbender', 'Is_Milla Jovovich', 'Is_Morgan Freeman', 'Is_Naomi Watts', 'Is_Natalie Portman', 'Is_Nicolas Cage', 'Is_Nicole Kidman', 'Is_Other', 'Is_Patrick Stewart', 'Is_Paul Rudd', 'Is_Robert De Niro', 'Is_Robert Downey Jr.', 'Is_Robin Williams', 'Is_Russell Crowe', 'Is_Ryan Gosling', 'Is_Ryan Reynolds', 'Is_Sam Worthington', 'Is_Samuel L. Jackson', 'Is_Sandra Bullock', 'Is_Scarlett Johansson', 'Is_Shailene Woodley', 'Is_Shia LaBeouf', 'Is_Steve Carell', 'Is_Sylvester Stallone', 'Is_Tobey Maguire', 'Is_Tom Cruise', 'Is_Tom Hanks', 'Is_Tom Hardy', 'Is_Tom Holland', 'Is_Tommy Lee Jones', 'Is_Uma Thurman', 'Is_Vin Diesel', 'Is_Will Ferrell', 'Is_Will Smith'
       ]

In [83]:
le_realisateur = LabelEncoder()
le_acteur = LabelEncoder()
le_producteur = LabelEncoder()
le_popularity = LabelEncoder()
le_vote_avergage = LabelEncoder()
le_vote_count = LabelEncoder()
# le_film = LabelEncoder()

# RECO['titre_FR'] = le_film.fit_transform(RECO['Titre_FR'])
RECO['DirectorName'] = le_realisateur.fit_transform(RECO['DirectorName'])
RECO['Actor_Name'] = le_acteur.fit_transform(RECO['Actor_Name'])
RECO['prod_1'] = le_producteur.fit_transform(RECO['prod_1'])
RECO['popularity'] = le_producteur.fit_transform(RECO['popularity'])
RECO['vote_average'] = le_producteur.fit_transform(RECO['vote_average'])
RECO['vote_count'] = le_vote_count.fit_transform(RECO['vote_count'])

In [84]:
RECO.describe()

,DirectorName,Actor_Name,runtime,release_date,prod_1,vote_average,vote_count,popularity,id,bonus_animation,...,bonus_Horror,bonus_Scifi,bonus_War,bonus_Western,bonus_Disney,bonus_Pixar,bonus_Dreamworks,bonus_Ghibli,bonus_Lucasfilm,bonus_Marvel
count,27282.000000,27282.000000,27282.000000,27219,27282.000000,27282.000000,27282.000000,27282.000000,2.728200e+04,27282.000000,...,27282.000000,27282.000000,27282.000000,27282.000000,27282.000000,27282.000000,27282.000000,27282.000000,27282.000000,27282.000000
mean,33.857892,71.434975,97.814603,2011-11-29 09:32:50.814504704,27.003299,1618.337659,323.420497,4966.291181,3.132093e+05,0.032806,...,0.068104,0.018840,0.007661,0.002639,0.004912,0.000733,0.001136,0.000477,0.000367,0.002346
min,0.000000,0.000000,0.000000,1989-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,5.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34.000000,73.000000,89.000000,2005-08-17 12:00:00,30.000000,859.000000,9.000000,1299.000000,4.475175e+04,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,34.000000,73.000000,97.000000,2013-03-06 00:00:00,30.000000,1632.000000,44.000000,4135.000000,2.217665e+05,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,34.000000,73.000000,108.000000,2018-12-12 00:00:00,30.000000,2329.000000,246.000000,8036.750000,5.225240e+05,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,50.000000,100.000000,803.000000,2023-12-08 00:00:00,50.000000,3633.000000,3248.000000,13899.000000,1.121898e+06,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
std,2.375156,9.480810,24.073929,NaN,8.295467,943.623539,646.775741,4040.915180,3.020516e+05,0.178130,...,0.251928,0.135963,0.087191,0.051305,0.069912,0.027066,0.033690,0.021824,0.019142,0.068457


In [85]:
X = RECO[ColModel]

y = RECO['Titre_FR']

In [86]:
RECO.head()

,imdb_id,original_title,Titre_FR,DirectorName,Actor_Name,overview,runtime,release_date,prod_1,production_companies_country,...,bonus_Horror,bonus_Scifi,bonus_War,bonus_Western,bonus_Disney,bonus_Pixar,bonus_Dreamworks,bonus_Ghibli,bonus_Lucasfilm,bonus_Marvel
0,tt0113101,Four Rooms,Groom Service,34,73,It's Ted the Bellhop's first night on the job....,98,1995-12-09,26,"['US', 'US']",...,0,0,0,0,0,0,0,0,0,0
1,tt0266543,Finding Nemo,Le Monde de Nemo,34,73,"Nemo, an adventurous young clownfish, is unexp...",100,2003-05-30,32,['US'],...,0,0,0,0,0,1,0,0,0,0
2,tt0169547,American Beauty,American Beauty,42,56,"Lester Burnham, a depressed suburban father in...",122,1999-09-15,30,"['', 'US']",...,0,0,0,0,0,0,0,0,0,0
3,tt0168629,Dancer in the Dark,Dancer in the Dark,34,73,"Selma, a Czech immigrant on the verge of blind...",141,2000-06-30,30,"['', '', 'DK', 'DK', 'SE', 'FR', 'SE', 'FR', '...",...,0,0,0,0,0,0,0,0,0,0
4,tt0411267,The Dark,The Dark,34,73,"In an attempt to pull her family together, Adè...",87,2005-09-28,30,"['DE', 'GB', 'GB', 'GB']",...,1,0,0,0,0,0,0,0,0,0


In [87]:
RECO.shape

(27282, 250)

In [88]:
import pandas as pd

# Assuming df is your DataFrame
duplicates = RECO.columns[RECO.columns.duplicated()].unique()
print("Duplicate columns:", duplicates)

Duplicate columns: Index(['Is_Other'], dtype='object')


In [89]:
RECO = RECO.drop(columns="Is_Other")

In [90]:
RECO.to_csv("C:\\Users\\nicol\\Desktop\\Data\\VSC\\repos\\Wild Code School\\projet\\P2 - IMDB\\Recommandation\\RECO_v2.csv")

4- Choix du modèle

In [91]:
# Choix du modèle de prédiction (régression, classification, clustering)

from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=6, metric='euclidean')

5- Entrainement du modèle

In [92]:
#  ajustement des paramètres avec des algorithmes d’optimisation.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


nn.fit(X_scaled)

NearestNeighbors(metric='euclidean', n_neighbors=6)

6- Evaluation du modèle

In [93]:
# mesure de la performance avec des métriques (précision, rappel, RMSE).

# Les modèles non supervisés ne nécessitent pas d'évaluation

7- Optimisation du modèle

In [94]:
# ajustement des hyperparamètres pour améliorer la performance.

8- TEST

In [95]:
RECO

,imdb_id,original_title,Titre_FR,DirectorName,Actor_Name,overview,runtime,release_date,prod_1,production_companies_country,...,bonus_Horror,bonus_Scifi,bonus_War,bonus_Western,bonus_Disney,bonus_Pixar,bonus_Dreamworks,bonus_Ghibli,bonus_Lucasfilm,bonus_Marvel
0,tt0113101,Four Rooms,Groom Service,34,73,It's Ted the Bellhop's first night on the job....,98,1995-12-09,26,"['US', 'US']",...,0,0,0,0,0,0,0,0,0,0
1,tt0266543,Finding Nemo,Le Monde de Nemo,34,73,"Nemo, an adventurous young clownfish, is unexp...",100,2003-05-30,32,['US'],...,0,0,0,0,0,1,0,0,0,0
2,tt0169547,American Beauty,American Beauty,42,56,"Lester Burnham, a depressed suburban father in...",122,1999-09-15,30,"['', 'US']",...,0,0,0,0,0,0,0,0,0,0
3,tt0168629,Dancer in the Dark,Dancer in the Dark,34,73,"Selma, a Czech immigrant on the verge of blind...",141,2000-06-30,30,"['', '', 'DK', 'DK', 'SE', 'FR', 'SE', 'FR', '...",...,0,0,0,0,0,0,0,0,0,0
4,tt0411267,The Dark,The Dark,34,73,"In an attempt to pull her family together, Adè...",87,2005-09-28,30,"['DE', 'GB', 'GB', 'GB']",...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27277,tt0114497,Sommaren,Sommaren,34,73,Torun and Mikael have been married for a few y...,112,1995-08-31,30,Donnée non disponible,...,0,0,0,0,0,0,0,0,0,0
27278,tt1728240,Ish lelo selolari,Téléphone arabe,34,73,Description non disponible,0,2012-07-25,9,Donnée non disponible,...,0,0,0,0,0,0,0,0,0,0
27279,tt1725065,Out Loud,Crie-le haut et fort,34,73,"Six friends (three men, one gay and a woman) g...",102,2011-10-04,9,Donnée non disponible,...,0,0,0,0,0,0,0,0,0,0
27280,tt0366808,Nachbarinnen,Les voisines,34,73,The single package-delivery woman Dora lives a...,88,2005-04-28,30,Donnée non disponible,...,0,0,0,0,0,0,0,0,0,0


In [96]:
Film_Test = "Toy Story 4"

print(RECO.index[RECO['Titre_FR'] == Film_Test])
print(TMDB_complete.index[TMDB_complete['Titre_FR'] == Film_Test])
# print(TMDB_complete.index[TMDB_complete['title'] == Film_Test])
print(TMDB.index[TMDB['original_title'] == Film_Test])
print(AKAS.index[AKAS['title'] == Film_Test])

Index([25862], dtype='int64')
Index([25862], dtype='int64')
Index([29330], dtype='int64')
Index([26901], dtype='int64')


In [97]:
# Recherche des films similaires àau film_Test

import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # Mets le nombre de cœurs logiques souhaité

film_index = RECO[RECO['Titre_FR'] == Film_Test].index[0]
distances, indices = nn.kneighbors(X_scaled[film_index].reshape(1, -1))

print(f"\nFilms les plus similaires à {Film_Test}:\n")
for distance, index in zip(distances[0][1:], indices[0][1:]):  # On exclut le film lui-même
    print(f"{RECO['Titre_FR'].iloc[index]} (Distance: {distance:.2f})")


Films les plus similaires à Toy Story 4:

Cloud Atlas (Distance: 16.70)
Elvis (Distance: 16.75)
La Mission (Distance: 16.89)
Le Pire voisin au monde (Distance: 16.99)
La Guerre selon Charlie Wilson (Distance: 17.02)


INTEGRATION A STREAMLIT

In [98]:
import streamlit as st
import random
from unidecode import unidecode

In [99]:
st.title("🎬 Recommandation de Films")

# Barre de recherche
Film_Test = st.text_input("Entrez le titre d'un film :")

if Film_Test:
    # Exécuter la requête de recommandation
    film_index = RECO[RECO['Titre_FR'] == Film_Test].index[0]
    distances, indices = nn.kneighbors(X_scaled[film_index].reshape(1, -1))

    st.subheader(f"Films les plus similaires à **{Film_Test}** :")
    for distance, index in zip(distances[0][1:], indices[0][1:]):  # On exclut le film lui-même
        st.write(f"{RECO['Titre_FR'].iloc[index]} (Distance: {distance:.2f})")

2025-05-09 18:02:25.402 
  command:

    streamlit run c:\Users\nicol\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-09 18:02:25.403 Session state does not function when running a script without `streamlit run`


In [100]:
# movie_list = RECO["Titre_FR"].tolist()

# # 🔎 Fonction pour suggérer des films (saisie semi-automatique)
# def get_movie_suggestions(user_input, movie_list):
#     if not user_input:
#         return []
#     user_input_norm = unidecode(user_input.lower())  # Supprime les accents
#     return [movie for movie in movie_list if unidecode(movie.lower()).startswith(user_input_norm)][:10]

# # 🎭 Sidebar pour la recherche et les filtres
# st.sidebar.title("🎬 Les Recos de Nico")
# st.sidebar.write("Nico vous recommande 5 films à partir d'une sélection !")

# user_movie_input = st.sidebar.text_input("Choisissez un film :")

# # 🎭 Filtre par genre
# genres = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 
#           'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 
#           'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']
# selected_genre = st.sidebar.selectbox("Filtrer par genre :", genres)

# # 🔍 Suggestions de films
# suggestions = get_movie_suggestions(user_movie_input, movie_list)
# selected_movie = st.sidebar.radio("Sélectionnez un film:", suggestions) if suggestions else None

# # 📌 Filtrage des films par genre
# filtered_movies = RECO[RECO[selected_genre] == 1] if selected_genre else RECO

# # 🎥 Affichage du film sélectionné
# if selected_movie:
#     st.write(f"Film sélectionné : **{selected_movie}**")

# # 🔥 Fonction pour obtenir les recommandations
# def get_recommendations(movie_title):
#     if movie_title not in RECO["Titre_FR"].values:
#         st.warning("⚠️ Film introuvable dans la base de données.")
#         return []

#     movie_index = RECO[RECO["Titre_FR"] == movie_title].index[0]
#     distances, indices = nn.kneighbors([X_scaled[movie_index]])

#     recommendations = []
#     for idx in indices[0][1:]:  # Exclure le film sélectionné lui-même
#         recommendations.append(RECO.iloc[idx])

#     return recommendations

# # 🌟 Affichage des recommandations
# BASE_URL = "https://image.tmdb.org/t/p/w500/"

# if selected_movie:
#     recommendations = get_recommendations(selected_movie)

#     if recommendations:
#         st.subheader("📌 Ces films peuvent également vous intéresser :")
#         cols = st.columns(min(len(recommendations), 5))
#         for i, movie in enumerate(recommendations[:5]):
#             with cols[i]:
#                 poster_path = movie["poster_path"] if pd.notna(movie["poster_path"]) else None
#                 poster_url = BASE_URL + poster_path if poster_path else "https://via.placeholder.com/150"
#                 st.image(poster_url, width=150)
#                 st.write(f"**{movie['Titre_FR']}**")
#     else:
#         st.write("Désolé, aucun film similaire trouvé.")

# # 🎲 Affichage des films aléatoires si aucune recherche n'est effectuée
# if not selected_movie:
#     st.subheader("🎬 Films générés aléatoirement :")
#     random_movies = random.sample(list(filtered_movies['Titre_FR']), min(10, len(filtered_movies)))

#     cols = st.columns(5)
#     for i, movie in enumerate(random_movies[:5]):
#         with cols[i]:
#             movie_row = RECO[RECO["Titre_FR"] == movie]
#             poster_path = movie_row["poster_path"].values[0] if not movie_row.empty and pd.notna(movie_row["poster_path"].values[0]) else None
#             poster_url = BASE_URL + poster_path if poster_path else "https://via.placeholder.com/150"
#             st.image(poster_url, width=150)
#             st.write(f"**{movie}**")

#     cols = st.columns(5)
#     for i, movie in enumerate(random_movies[5:]):
#         with cols[i]:
#             movie_row = RECO[RECO["Titre_FR"] == movie]
#             poster_path = movie_row["poster_path"].values[0] if not movie_row.empty and pd.notna(movie_row["poster_path"].values[0]) else None
#             poster_url = BASE_URL + poster_path if poster_path else "https://via.placeholder.com/150"
#             st.image(poster_url, width=150)
#             st.write(f"**{movie}**")